In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Reshape, Activation, BatchNormalization

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/sudoku.csv')
df.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [12]:
df['quizzes'].shape[0]

1000000

In [47]:
def generate_matrix(string):
    puzzle = np.array([list(x) for x in string],dtype='int').reshape(string.shape[0],9,9,1)
    return puzzle
def gen_sol_matrix(string):
    puzzle = np.array([list(x) for x in string],dtype='int')
    return puzzle
def norm(a):
    return (a/9)-.5
def denorm(a):
    return (a+.5)*9

In [48]:
quiz_matrix = norm(generate_matrix(df['quizzes']))
sol_matrix = gen_sol_matrix(df['solutions'])

In [ ]:
np.array([x for x in df['quizzes'][0] ],dtype='int').reshape(3,3,3,3), "----------------------------------" ,sol_matrix[0].reshape(3,3,3,3)

In [50]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 9, 9, 64)          640       
                                                                 
 batch_normalization_10 (Bat  (None, 9, 9, 64)         256       
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_11 (Bat  (None, 9, 9, 64)         256       
 chNormalization)                                                
                                                                 
 conv2d_8 (Conv2D)           (None, 9, 9, 128)         8320      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 128)       

In [51]:
model.compile(optimizer="adam", loss= tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

In [54]:
history = model.fit(quiz_matrix,
          sol_matrix-1,
          batch_size=128,
          epochs=50,
                    validation_split=0.2,
          shuffle=True)

Epoch 1/50
6250/6250 [==============================] - 34s 5ms/step - loss: 0.3393 - accuracy: 0.8427 - val_loss: 0.3528 - val_accuracy: 0.8346
Epoch 2/50
6250/6250 [==============================] - 34s 5ms/step - loss: 0.3393 - accuracy: 0.8428 - val_loss: 0.3528 - val_accuracy: 0.8347
Epoch 3/50
6250/6250 [==============================] - 34s 5ms/step - loss: 0.3392 - accuracy: 0.8428 - val_loss: 0.3527 - val_accuracy: 0.8346
Epoch 4/50
6250/6250 [==============================] - 34s 5ms/step - loss: 0.3391 - accuracy: 0.8429 - val_loss: 0.3524 - val_accuracy: 0.8348
Epoch 5/50
6250/6250 [==============================] - 33s 5ms/step - loss: 0.3391 - accuracy: 0.8429 - val_loss: 0.3535 - val_accuracy: 0.8344
Epoch 6/50
6250/6250 [==============================] - 34s 5ms/step - loss: 0.3390 - accuracy: 0.8429 - val_loss: 0.3536 - val_accuracy: 0.8343
Epoch 7/50
6250/6250 [==============================] - 34s 5ms/step - loss: 0.3389 - accuracy: 0.8430 - val_loss: 0.3526 - val_ac